In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from mlxtend.frequent_patterns import apriori, association_rules 

C:\Users\HP\anaconda3\lib\site-packages\seaborn\rcmod.py:400: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
C:\Users\HP\anaconda3\lib\site-packages\setuptools\_distutils\version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


# Basket Analysis/Association Rule Mining of Visits

In [3]:
df_raw = pd.read_excel("Overall_Raw_Adj.xls")

In [4]:
df_raw

,Insurance,Patient_Id,Date,Procedure,Units,Charge_Resp,Charge,Contractual_Adj,Charge Balance
0,Absolute Total Care Medicaid,72181,1/2/2020,90460,1,P,22.0,-1.84,0.0
1,Absolute Total Care Medicaid,72181,1/2/2020,90461,2,P,40.0,-16.24,0.0
2,Absolute Total Care Medicaid,72181,1/2/2020,99391,1,P,125.0,-28.41,0.0
3,Absolute Total Care Medicaid,72508,1/8/2020,99381,1,P,151.0,-43.87,0.0
4,Absolute Total Care Medicaid,72307,1/10/2020,90460,1,P,22.0,-1.84,0.0
...,...,...,...,...,...,...,...,...,...
36516,Valenz Tucker McClancy,69779,10/18/2021,90471,1,P,25.0,0.00,0.0
36517,Valenz Tucker McClancy,69779,10/18/2021,90686,1,P,30.0,0.00,0.0
36518,Valenz Tucker McClancy,69779,10/18/2021,99393,1,P,160.0,0.00,0.0
36519,WellCare Health Medicaid,72640,4/20/2020,99381,1,P,151.0,-67.95,0.0


### Add Visit ID column

In [5]:
patients = df_raw['Patient_Id'].apply(str)
dates = df_raw['Date'].apply(str)
visit_id = dict()
num = 1
col = []
for patient, date in zip(patients, dates):
    temp = patient + date
    visit_id[temp] = visit_id.get(temp,num)
    col.append(visit_id.get(temp))
    temp = ''
    num+=1
df_raw['Visit_Id'] = col

In [6]:
df_raw

,Insurance,Patient_Id,Date,Procedure,Units,Charge_Resp,Charge,Contractual_Adj,Charge Balance,Visit_Id
0,Absolute Total Care Medicaid,72181,1/2/2020,90460,1,P,22.0,-1.84,0.0,1
1,Absolute Total Care Medicaid,72181,1/2/2020,90461,2,P,40.0,-16.24,0.0,1
2,Absolute Total Care Medicaid,72181,1/2/2020,99391,1,P,125.0,-28.41,0.0,1
3,Absolute Total Care Medicaid,72508,1/8/2020,99381,1,P,151.0,-43.87,0.0,4
4,Absolute Total Care Medicaid,72307,1/10/2020,90460,1,P,22.0,-1.84,0.0,5
...,...,...,...,...,...,...,...,...,...,...
36516,Valenz Tucker McClancy,69779,10/18/2021,90471,1,P,25.0,0.00,0.0,36517
36517,Valenz Tucker McClancy,69779,10/18/2021,90686,1,P,30.0,0.00,0.0,36517
36518,Valenz Tucker McClancy,69779,10/18/2021,99393,1,P,160.0,0.00,0.0,36517
36519,WellCare Health Medicaid,72640,4/20/2020,99381,1,P,151.0,-67.95,0.0,36520


### Group by Visit Id

In [7]:
df_raw.groupby(by=['Visit_Id', 'Procedure'], as_index=False)

In [8]:
df = df_raw.groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')

In [9]:
df_mb = df.applymap(lambda x: False if x<=0 else True)

In [10]:
df_mb

Procedure,10060,10120,11981,17250,41010,51701,54150,69090,69096,69200,...,J7510,J7512,J7611,J7613,J7619,J7620,LFEE,MDREC,NSF,PMPM
Visit_Id,,,,,,,,,,,,,,,,,,,,,
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36515,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36516,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
36517,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Total Association Rule Mining

In [ ]:
freq_itemsets = apriori(df_mb, min_support=0.05, use_colnames=True)
freq_itemsets

In [ ]:
rules = association_rules(freq_itemsets, metric='lift', min_threshold=1)
rules.sort_values(by=['confidence','support'], ascending=False)

### Association Rule Mining for Most frequent 6 Insurances (31,295 data points or 85.7% of total points)

In [ ]:
df_raw.Insurance.value_counts()

In [ ]:
df_top_ins = df_raw.groupby('Insurance').filter(lambda x: len(x)>1000)

#### BCBS - PPO Plans

In [ ]:
df_bcbsppo = df_top_ins[df_top_ins['Insurance']=='BCBS - PPO Plans'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_bcbsppo = df_bcbsppo.applymap(lambda x: False if x<=0 else True)
freq_itemsets_bcbsppo = apriori(df_bcbsppo, min_support=0.05, use_colnames=True)
freq_itemsets_bcbsppo

In [ ]:
rules_bcbsppo = association_rules(freq_itemsets_bcbsppo, metric='lift', min_threshold=1)
rules_bcbsppo.sort_values(by=['confidence','support'], ascending=False)

#### Molina Healthcare Medicaid

In [ ]:
df_mhm = df_top_ins[df_top_ins['Insurance']=='Molina Healthcare Medicaid'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_mhm = df_mhm.applymap(lambda x: False if x<=0 else True)
freq_itemsets_mhm = apriori(df_mhm, min_support=0.05, use_colnames=True)
freq_itemsets_mhm

In [ ]:
rules_mhm = association_rules(freq_itemsets_mhm, metric='lift', min_threshold=1)
rules_mhm.sort_values(by=['confidence','support'], ascending=False)

#### BCBS SC SHP

In [ ]:
df_bcbssc = df_top_ins[df_top_ins['Insurance']=='BCBS SC SHP'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_bcbssc = df_bcbssc.applymap(lambda x: False if x<=0 else True)
freq_itemsets_bcbssc = apriori(df_bcbssc, min_support=0.05, use_colnames=True)
freq_itemsets_bcbssc

In [ ]:
rules_bcbssc = association_rules(freq_itemsets_bcbssc, metric='lift', min_threshold=1)
rules_bcbssc.sort_values(by=['confidence','support'], ascending=False)

#### Healthy Blue Medicaid

In [ ]:
df_hbm = df_top_ins[df_top_ins['Insurance']=='Healthy Blue Medicaid'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_hbm = df_hbm.applymap(lambda x: False if x<=0 else True)
freq_itemsets_hbm = apriori(df_hbm, min_support=0.05, use_colnames=True)
freq_itemsets_hbm

In [ ]:
rules_hbm = association_rules(freq_itemsets_hbm, metric='lift', min_threshold=1)
rules_hbm.sort_values(by=['confidence','support'], ascending=False)

#### United Healthcare

In [ ]:
df_uh = df_top_ins[df_top_ins['Insurance']=='United Healthcare'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_uh = df_uh.applymap(lambda x: False if x<=0 else True)
freq_itemsets_uh = apriori(df_uh, min_support=0.05, use_colnames=True)
freq_itemsets_uh

In [ ]:
rules_uh = association_rules(freq_itemsets_uh, metric='lift', min_threshold=1)
rules_uh.sort_values(by=['confidence','support'], ascending=False)

#### Cigna

In [ ]:
df_c = df_top_ins[df_top_ins['Insurance']=='Cigna'].groupby(['Visit_Id','Procedure'])['Units'].sum().unstack().reset_index().fillna(0).set_index('Visit_Id')
df_c = df_c.applymap(lambda x: False if x<=0 else True)
freq_itemsets_c = apriori(df_c, min_support=0.05, use_colnames=True)
freq_itemsets_c

In [ ]:
rules_c = association_rules(freq_itemsets_c, metric='lift', min_threshold=1)
rules_c.sort_values(by=['confidence','support'], ascending=False)